In [161]:
%load_ext autoreload
%autoreload 2
import utils.dataset as dataset
import utils.preprocessing as preprocessing
from utils.logger import Logger
import lstm.model as model
import utils.postprocessing as postprocessing
import pandas as pd
from sklearn import metrics
from numpy import linalg as LA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch
import torch.nn as nn
from pathlib import Path
from torch.autograd import Variable

In [8]:
train_path= '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv'
test_path = '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv'

train_x, test_x, test_y = dataset.pca_return(train_path = train_path, test_path = test_path)

# SVM

In [27]:
from sklearn import svm

In [52]:
# comp_num = 45
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,0:2])
preds = clf.predict(test_x[5000:15000, 0:2]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 4842, 1: 5158}


In [46]:
# comp_num = 45
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,0:4])
preds = clf.predict(test_x[5000:15000, 0:4]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 2390, 1: 7610}


In [43]:
# comp_num = 2
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,:])
preds = clf.predict(test_x[5000:15000, :]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 4842, 1: 5158}


# LSTM

## model

In [10]:
class ENCODER(nn.Module):

    def __init__(self,args):
        super(ENCODER, self).__init__()
        self.args = args
        self.drop = nn.Dropout(args['dropout'])
        self.linear = nn.Linear(args['hidden_size'], args['data_dim'])

        if args['cell_type'] in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, args['cell_type'])(args['rnn_inp_size'], args['hidden_size'], args['nlayers'], dropout=args['dropout'])

    def init_weights(self):
        initrange = 0.1
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)

    def forward(self, input, hidden, return_hiddens=False, noise=False):
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output.contiguous().view(-1,self.args['hidden_size']))
        output = output.contiguous().view(input.size()[0], -1, self.args['rnn_inp_size'])
        return output, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data ############# 이게 무엇
        if self.args['cell_type'] == 'LSTM':
            return (Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()),
                    Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == tuple:
            return tuple(self.repackage_hidden(v) for v in h)
        else:
            return Variable(h.data)

    def extract_hidden(self, hidden):
        if self.args['cell_type'] == 'LSTM':
            return hidden[0][-1].data.cpu()  # hidden state last layer (hidden[1] is cell state)
        else:
            return hidden[-1].data.cpu()  # last layer

        
class DECODER(nn.Module):

    def __init__(self,args):
        super(DECODER, self).__init__()
        self.args = args
        self.drop = nn.Dropout(args['dropout'])
        self.linear = nn.Linear(args['hidden_size'], args['data_dim'])

        if args['cell_type'] in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, args['cell_type'])(args['rnn_inp_size'], args['hidden_size'], args['nlayers'], dropout=args['dropout'])


    def init_weights(self):
        initrange = 0.1
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)
        
    def forward(self, input, hidden, return_hiddens=False, noise=False):
        
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output.contiguous().view(-1,self.args['hidden_size']))
        output = output.contiguous().view(input.size()[0], -1, self.args['rnn_inp_size'])

        return output, hidden




    def init_hidden(self, bsz):
        weight = next(self.parameters()).data ############# 이게 무엇
        if self.args['cell_type'] == 'LSTM':
            return (Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()),
                    Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == tuple:
            return tuple(self.repackage_hidden(v) for v in h)
        else:
            return Variable(h.data)

    def extract_hidden(self, hidden):
        if self.args['cell_type'] == 'LSTM':
            return hidden[0][-1].data.cpu()  # hidden state last layer (hidden[1] is cell state)
        else:
            return hidden[-1].data.cpu()  # last layer

## solver

In [170]:
class Solver():
    def __init__(self, args):
        
        train_x, test_x, test_y = dataset.pca_return(train_path = args['train_path'], test_path = args['test_path'])
        
        train_x_batchfy = preprocessing.batchify(args, train_x, args['batch_size'])
        test_x_batchfy = preprocessing.batchify(args, test_x, args['batch_size'])
        generate_batchfy = preprocessing.batchify(args, test_x, 1)
        train_generate_batchfy = preprocessing.batchify(args, train_x, 1)
        
        self.train_x_batchfy = train_x_batchfy[:,:,args['selected_dim']]
        self.test_x_batchfy = test_x_batchfy[:,:,args['selected_dim']]
        self.generate_batchfy = generate_batchfy[:,:,args['selected_dim']]
        self.train_generate_batchfy = train_generate_batchfy[:,:,args['selected_dim']]
        self.test_y = test_y
        

        self.args = args
        self.encoder = model.ENCODER(self.args)
        self.encoder.cuda()

        self.decoder = model.DECODER(self.args)
        self.decoder.cuda()

        self.optim_enc   = torch.optim.Adam(self.encoder.parameters(), self.args['lr'])
        self.optim_dec   = torch.optim.Adam(self.decoder.parameters(), self.args['lr'])

        self.loss_fn = nn.MSELoss()    
    
        self.logger = Logger('./tf_logs')
    
        self.base_dir = Path('save',self.make_dir_name(args))
        self.base_dir.mkdir(parents=True,exist_ok=True)      
        
        self.evaluate = args['evaluate']
    
    def make_dir_name(self, args):
        return 'modelName:'+args['model_name']+'__cellType:'+args['cell_type'] \
                + '__hidSize:' + str(args['hidden_size']) + '__dropout:' + str(args['dropout'])

    def load(self, path):
        try:
            checkpoint = torch.load(Path(path))
            start_epoch = checkpoint['epoch']
            self.encoder.load_state_dict(checkpoint['state_dict_enc'])
            self.optim_enc.load_state_dict((checkpoint['optimizer_enc']))
            self.decoder.load_state_dict(checkpoint['state_dict_dec'])
            self.optim_dec.load_state_dict((checkpoint['optimizer_dec']))            
            del checkpoint
            print("=> loaded checkpoint")
        except:
            print("=> Not exist checkpoint")
            pass        

    def fit(self, load):
        if load == True:
            try:
                checkpoint = torch.load(Path(self.base_dir,'checkpoint').with_suffix('.pth'))
                start_epoch = checkpoint['epoch']
                self.encoder.load_state_dict(checkpoint['state_dict_enc'])
                self.optim_enc.load_state_dict((checkpoint['optimizer_enc']))
                self.decoder.load_state_dict(checkpoint['state_dict_dec'])
                self.optim_dec.load_state_dict((checkpoint['optimizer_dec']))            
                del checkpoint
                print("=> loaded checkpoint")
            except:
                print("=> Not exist checkpoint")
                pass

        total_loss = 0

        for epoch in range(0, self.args['epoch']):

            self.encoder.train()
            self.decoder.train()
            
            print("post in")
            self.anomal_score = postprocessing.get_anomalscore_minmax(base_model = self,  \
                                                                 generate_batchfy = self.generate_batchfy, length = 449916,args = self.args)
            print(anomal_score)
            self.anomal_score = LA.norm(self.anomal_score, axis=1)
            print(anomal_score)
            precision, recall, thresholds = metrics.precision_recall_curve(  self.test_y[:449916].values.astype(int), self.anomal_score,pos_label =1)
            beta = 1
            f1 = (1+beta**2)*(precision*recall)/((beta**2*precision)+recall)
            f1 = np.nan_to_num(f1)
            max_f1 = f1[np.argmax(f1)]
            print("precision : {}, recall : {}, f1 : {}".format(precision,recall, f1))
            break
                
            hidden_enc = self.encoder.init_hidden(self.args['batch_size'])

            for batch, i in enumerate(range(0, self.train_x_batchfy.size(0) - 1, self.args['seq_length'])):
                outSeq = []
                inputSeq, targetSeq = preprocessing.get_batch(self.args, self.train_x_batchfy, i)

                if args['seq_length'] != targetSeq.size()[0] :
                    continue
                hidden_enc = self.encoder.repackage_hidden(hidden_enc)
                self.optim_enc.zero_grad()
                self.optim_dec.zero_grad()
                
                Outputseq_enc, hidden_enc = self.encoder.forward(inputSeq, hidden_enc, return_hiddens=True)
                deccoder_input = Variable(torch.zeros(Outputseq_enc.size())).cuda()
                
                deccoder_input[0,:,:] = Outputseq_enc[-1,:,:] # inputSeq[-1,:,:]
                deccoder_input[1:,:,:] = targetSeq[:-1,:,:]
                
                loss_enc = self.loss_fn(Outputseq_enc[-1,:,:].view(self.args['batch_size'], -1), targetSeq[0,:,:].contiguous().view(self.args['batch_size'], -1))
                loss_enc.backward(retain_graph=True)
                
                total_norm_1 = max(p.grad.data.abs().max() for p in self.encoder.parameters())
                
                torch.nn.utils.clip_grad_norm(self.encoder.parameters(), 0.1)
                
                total_norm_2 = max(p.grad.data.abs().max() for p in self.encoder.parameters())
#                 break
                
#                 print("parameter norm {}, after clip {}".format(total_norm_1, total_norm_2))
#                 print(batch, self.train_x_batchfy.size(0) - 1)
#                 if batch > 10:
#                     break
                
                self.optim_enc.step()     
                
                Outputseq_enc, hidden_enc = self.decoder.forward(deccoder_input, hidden_enc, return_hiddens=True)
                loss_dec = self.loss_fn(Outputseq_enc.view(args['batch_size'], -1), targetSeq.contiguous().view(args['batch_size'], -1))   
                loss_dec.backward()
                torch.nn.utils.clip_grad_norm(self.decoder.parameters(), self.args['clip'])
                self.optim_dec.step()
                
                total_loss += loss_enc.item() + loss_dec.item()        

                if batch % 10 == 0 :
                    # 1. Log scalar values (scalar summary)
                    info = { 'enc_loss': loss_enc.item(), 'dec_loss' : loss_dec.item() }

#                     for tag, value in info.items():
#                         self.logger.scalar_summary(tag, value, step+1)

#                     # 2. Log values and gradients of the parameters (histogram summary)
#                     for tag, value in self.net.named_parameters():
#                         tag = tag.replace('.', '/')
#                         self.logger.histo_summary(tag, value.data.cpu().numpy(), step+1)
              
            
#             self.model_dictionary = {'epoch': epoch + 1,
#                     'state_dict_enc': self.encoder.state_dict(),
#                     'optimizer_enc': self.optim_enc.state_dict(),
#                     'state_dict_dec': self.decoder.state_dict(),
#                     'optimizer_dec': self.optim_dec.state_dict(),
#                     'args':args,
#                     'loss':total_loss
#                     }
#             self.save_checkpoint(self.args, self.model_dictionary)
            total_loss = 0
            anomal_score = postprocessing.get_anomalscore_encdec(base_model = self,  \
                                                                 generate_batchfy = self.generate_batchfy, length = 449916,args = self.args)
            

 
    def save_checkpoint(self, args, state):
        checkpoint = Path(self.base_dir, 'checkpoint')
        checkpoint = checkpoint.with_suffix('.pth')
        torch.save(state, checkpoint)
        

In [ ]:
postprocessing.evaluate_conv(nomalize_scores = anomal_score, conv = 3, check_step = ,prints= True, endpoint = 449916, args= args)

In [141]:
anomal_score

NameError: name 'anomal_score' is not defined

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(  args['test_y'][:449916].cpu().numpy(), anomal_score,pos_label =1)

In [12]:
args={}
args['model_name'] = 'enc_dec'
args['cuda'] = True
args['batch_size'] = 256
args['clip'] = 4

In [13]:
import utils.preprocessing as preprocessing
import utils.postprocessing as postprocessing

In [14]:
train_x_batchfy = preprocessing.batchify(args, train_x, args['batch_size'])
test_x_batchfy = preprocessing.batchify(args, test_x, args['batch_size'])
generate_batchfy = preprocessing.batchify(args, test_x, 1)
train_generate_batchfy = preprocessing.batchify(args, train_x, 1)

args['test_y'] = test_y
args['cell_type'] = 'LSTM'

In [15]:
train_col = 3
args['epoch'] = 1
args['seq_length'] = 2
args['hidden_size'] = 32
args['train_x_batchfy'] = train_x_batchfy[:,:,[0,1,2]]
args['test_x_batchfy'] = test_x_batchfy[:,:,[0,1,2]]
args['generate_batchfy'] =generate_batchfy[:,:,[0,1,2]]
args['train_generate_batchfy'] =train_generate_batchfy[:,:,[0,1,2]]
args['data_dim'] = train_col
args['dropout'] = 0.5
args['nlayers'] = 2
args['rnn_inp_size'] = args['data_dim']

In [20]:
args['lr'] =0.001
solver = Solver(args = args)
solver.fit(load = False)

0 1939
1 1939
2 1939
3 1939
4 1939
5 1939
6 1939
7 1939
8 1939
9 1939
10 1939
11 1939


/home/mjh319/anaconda3/envs/latest_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:89: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/home/mjh319/anaconda3/envs/latest_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


In [107]:
import utils.postprocessing as postprocessing
anomal_score = postprocessing.get_anomalscore_encdec(base_model = solver,  generate_batchfy = solver.generate_batchfy, length = 449916,args = args)
# _,_,_ = utils.evaluate_conv(nomalize_scores = anomal_score ,num_samples = 100000, conv = 0, check_step = 1000, attack_list = attack_list, length = 449916,prints=True,args = args)

anomal_detection...


In [109]:
anomal_score.shape

(449916, 3)

In [25]:
# parameter
train_path= '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv'
test_path = '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv'
train_x, test_x, test_y = dataset.pca_return(train_path = train_path, test_path = test_path)

attack_list = '../../DATA/SWaT/SWaT_Physical/attack_list.csv'
attack_list = pd.read_csv(attack_list, error_bad_lines=False, sep='\t')

/home/mjh319/workspace/swat_anomal/utils/dataset.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_y[test_y == 'Attack'] = 1
/home/mjh319/workspace/swat_anomal/utils/dataset.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_y[test_y == 'Normal'] = 0


In [ ]:
# parameter
train_path= '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv'
test_path = '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv'
attack_list = '../../DATA/SWaT/SWaT_Physical/attack_list.csv'
attack_list = pd.read_csv(attack_list, error_bad_lines=False, sep='\t')

args={}
args['model_name'] = 'enc_dec'
args['cuda'] = True
args['batch_size'] = 256
args['clip'] = 4
args['train_path'] = train_path
args['test_path'] = test_path

train_x, test_x, test_y = dataset.pca_return(train_path = train_path, test_path = test_path)

train_x_batchfy = utils.batchify(args, train_x, args['batch_size'])
test_x_batchfy = utils.batchify(args, test_x, args['batch_size'])
generate_batchfy = utils.batchify(args, test_x, 1)
train_generate_batchfy = utils.batchify(args, train_x, 1)

args['test_y'] = test_y
args['cell_type'] = 'LSTM'


In [118]:
train_path= '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv'
test_path = '../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv'
attack_list = '../../DATA/SWaT/SWaT_Physical/attack_list.csv'
attack_list = pd.read_csv(attack_list, error_bad_lines=False, sep='\t')

args={}
args['model_name'] = 'enc_dec'
args['cuda'] = True
args['batch_size'] = 256
args['clip'] = 4
args['train_path'] = train_path
args['test_path'] = test_path
args['dropout'] = 0.5
args['hidden_size'] = 2
args['data_dim'] = 2
args['rnn_inp_size'] = args['data_dim']
args['nlayers'] = 2
args['lr'] =0.001
args['cell_type'] = 'LSTM'
args['epoch'] = 1
args['seq_length'] = 2
args['selected_dim'] = [0, 39]
args['evaluate'] = True

In [171]:
solver = Solver(args = args)

/home/mjh319/workspace/swat_anomal/utils/dataset.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_y[test_y == 'Attack'] = 1
/home/mjh319/workspace/swat_anomal/utils/dataset.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_y[test_y == 'Normal'] = 0


In [172]:
solver.fit(load = False)

post in


UnboundLocalError: local variable 'anomal_score' referenced before assignment

In [173]:
solver.anomal_score

array([[41.15925   , -0.89002657],
       [41.0561    , -0.9947951 ],
       [41.299503  , -0.87507385],
       ...,
       [60.857098  , -0.9810451 ],
       [61.40289   , -0.87138444],
       [61.910927  , -0.98094594]], dtype=float32)

In [174]:
anomal_score = LA.norm(solver.anomal_score, axis=1)

In [185]:
precision, recall, thresholds = metrics.precision_recall_curve(  solver.test_y[:449916].values.astype(int), anomal_score,pos_label =1)
beta = 1
f1 = (1+beta**2)*(precision*recall)/((beta**2*precision)+recall)
f1 = np.nan_to_num(f1)

In [186]:
f1

array([0.2165193 , 0.21651577, 0.21651223, ..., 0.        , 0.        ,
       0.        ])

In [187]:
max_f1 = f1[np.argmax(f1)]

In [188]:
max_f1

0.6225012156845409

In [184]:
t1.shape

(444639,)

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(  self.test_y[:449916].values.astype(int), self.anomal_score,pos_label =1)
beta = 1
f1 = (1+beta**2)*(precision*recall)/((beta**2*precision)+recall)
f1 = np.nan_to_num(f1)

In [179]:
type(solver.test_y[:449916].values)

numpy.ndarray

In [166]:
solver.test_y.values

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [152]:
np.unique(solver.test_y)

array([0, 1], dtype=object)

In [ ]:
args['hidden_size'] = 2
candi_front = [0, 1, 2]
candi_end = [38,39,40,41]
for i in range(len(candi_front)):
    for j in range(len(candi_end)):
        first = candi_front[i]
        second = candi_end[j]
        print("first dim = {}, end dim = {}".format(first, second ))
        train_col = 2
        args['epoch'] = 25
        args['seq_length'] = 2
        args['train_x_batchfy'] = train_x_batchfy[:,:,[first,second]]
        args['test_x_batchfy'] = test_x_batchfy[:,:,[first,second]]
        args['generate_batchfy'] =generate_batchfy[:,:,[first,second]]
        args['train_generate_batchfy'] =train_generate_batchfy[:,:,[first,second]]
        args['data_dim'] = train_col
        args['dropout'] = 0.5
        args['nlayers'] = 2
        args['rnn_inp_size'] = args['data_dim']
        args['lr'] =0.001
        solver = Solver(args = args)
        
#         try:
#             checkpoint = torch.load(Path(str(k) +','+str(k+1) +', enc, hid='+str(2)).with_suffix('.pth'))
#             test.encoder.load_state_dict(checkpoint['state_dict_enc'])
#             test.optim_enc.load_state_dict((checkpoint['optimizer_enc']))
#             test.decoder.load_state_dict(checkpoint['state_dict_dec'])
#             test.optim_dec.load_state_dict((checkpoint['optimizer_dec']))    
#         except:
#             print("not load")

        solver.fit(load = False)

        anomal_score_test = utils.get_anomalscore_encdec(base_model = test,  generate_batchfy = test.generate_batchfy, length = 449916,args = args)
        for conv in range(3):
            evaluate_conv2(nomalize_scores = anomal_score_test, conv = conv_candi[conv], prints= True, endpoint = 449916, args= args)

        model_dictionary = {'state_dict_enc': test.encoder.state_dict(),
        'optimizer_enc': test.optim_enc.state_dict(),
        'state_dict_dec': test.decoder.state_dict(),
        'optimizer_dec': test.optim_dec.state_dict(),
        'args':args,
        'anomal_score':anomal_score_test
        }
        checkpoint = Path(str(k) +','+str(k+1)+', enc, hid='+str(args['hidden_size']))
        checkpoint = checkpoint.with_suffix('.pth')
        torch.save(model_dictionary, checkpoint)

In [86]:
outSeq = []
scores = []
solver.encoder.eval()
solver.decoder.eval()
hidden_enc = solver.encoder.init_hidden(1)
endPoint = 2000 #446000 
step = args['seq_length']
feature_dim = generate_batchfy.size(-1)

In [87]:
for i in range(0,endPoint,step):
    Outputseq_enc, hidden_enc = solver.encoder.forward(Variable(args['generate_batchfy'] [i:i+step]).cuda(), hidden_enc)
    deccoder_input = Variable(torch.zeros(Outputseq_enc.size())).cuda()
    deccoder_input[0,:,:] = Outputseq_enc[-1,:,:]
    try:
        deccoder_input[1:,:,:] = args['generate_batchfy'] [i+step:i+step+step-1,:,:]
    except:
        continue
    Outputseq_enc, hidden_enc = solver.decoder.forward(deccoder_input, hidden_enc, return_hiddens=True)

    if Outputseq_enc.size()[0] != args['generate_batchfy'] [i:i+step].size()[0] :
        continue
    error = torch.abs(torch.add( Outputseq_enc.view(-1,args['generate_batchfy'] .size(-1)).cpu(), \
                            -args['generate_batchfy'] [i:i+step].view(-1,args['generate_batchfy'] .size(-1)).cpu()))       
    outSeq.append(error)

In [88]:
outSeq = torch.cat(outSeq,dim=0)

In [94]:
outSeq

tensor([[1.0169, 1.3731],
        [1.0903, 1.3254],
        [1.0688, 1.3496],
        ...,
        [0.8019, 0.9033],
        [0.7627, 0.9396],
        [0.7892, 0.9052]], grad_fn=<CatBackward>)

In [127]:
outSeq2 = outSeq.detach().numpy()

In [129]:
outSeq2.shape

(2000, 2)

In [134]:
outSeq2.min(axis=0)

0.00018640503

In [135]:
outSeq2 - outSeq2.min(axis=0) / (outSeq2.max(axis=0) - outSeq2.min(axis=0) )

array([[1.0167469 , 1.2815422 ],
       [1.0901258 , 1.2338567 ],
       [1.0687114 , 1.2580656 ],
       ...,
       [0.8017858 , 0.8117337 ],
       [0.7625318 , 0.84806687],
       [0.78907084, 0.8136948 ]], dtype=float32)

In [110]:
outSeq - outSeq.min(dim=0)[0] / (outSeq.max(dim=0)[0]  - outSeq.min(dim=0)[0] )

tensor([[1.0167, 1.2815],
        [1.0901, 1.2339],
        [1.0687, 1.2581],
        ...,
        [0.8018, 0.8117],
        [0.7625, 0.8481],
        [0.7891, 0.8137]], grad_fn=<ThSubBackward>)

In [97]:
outSeq.min(dim=0)

(tensor([0.0002, 0.5333], grad_fn=<MinBackward0>), tensor([897, 709]))

In [ ]:
for i in range(0,endPoint,step):
    Outputseq_enc, hidden_enc = base_model.encoder.forward(Variable(generate_batchfy[i:i+step]).cuda(), hidden_enc)
    deccoder_input = Variable(torch.zeros(Outputseq_enc.size())).cuda()
    deccoder_input[0,:,:] = Outputseq_enc[-1,:,:]
    try:
        deccoder_input[1:,:,:] = generate_batchfy[i+step:i+step+step-1,:,:]
    except:
        continue
    Outputseq_enc, hidden_enc = base_model.decoder.forward(deccoder_input, hidden_enc, return_hiddens=True)
    if Outputseq_enc.size()[0] != generate_batchfy[i:i+step].size()[0] :
        continue
    error = torch.abs(torch.add( Outputseq_enc.view(-1,generate_batchfy.size(-1)).cpu(), \
                            -generate_batchfy[i:i+step].view(-1,generate_batchfy.size(-1)).cpu()))       
    outSeq.append(error)

# test

In [110]:
import logging as log

In [113]:
from logging import handlers
import logging

In [111]:
log.basicConfig(filename='./log.txt', level=log.DEBUG)

log.debug('debug')
log.info('info')
log.warning('warning')
log.error('error')
log.critical('critical')

In [122]:
carLogFormatter = log.Formatter('%(asctime)s,%(message)s')

AttributeError: 'Logger' object has no attribute 'Formatter'

In [118]:
#handler settings
carLogHandler = handlers.TimedRotatingFileHandler(filename='car.log', interval=1, encoding='utf-8')
carLogHandler.setFormatter(carLogFormatter)
# carLogHandler.suffix = "%Y%m%d"

#logger set
carLogger = logging.getLogger()
carLogger.setLevel(logging.INFO)
carLogger.addHandler(carLogHandler)

#use logger
carLogger.info("car is coming22")

In [123]:
carLogFormatter = logging.Formatter('%(asctime)s,%(message)s')
log = logging.getLogger('snowdeer_log')
log.setLevel(logging.DEBUG)

fileHandler = logging.FileHandler('./log.txt')
fileHandler.setFormatter(carLogFormatter)
streamHandler = logging.StreamHandler()


log.addHandler(fileHandler)
log.addHandler(streamHandler)

In [124]:
log.info('info')

info
info
